In [ ]:
import pandas as pd

In [ ]:
s = pd.read_csv('cleaned.csv')

In [ ]:
s = s.drop(['Unnamed: 0'],axis=1)
#s3 = s3.loc[:,s3.columns!='target']
X =s.drop(['WORKSITE_STATE', 'WORKSITE_COUNTY_BIN','H1B_DEPENDENT',
       'AGENT_ATTORNEY_STATE_BIN', 'AMENDED_PETITION_BIN','NAICS_CODE_BIN',
       'AGENT_REPRESENTING_EMPLOYER','WAGE_UNIT_OF_PAY','EMPLOYER_CITY_BIN',
       'NEW_EMPLOYMENT_BIN', 'CHANGE_PREVIOUS_EMPLOYMENT_BIN','Visa_duration_bin','Target'],axis=1)
#X=X.drop(['Target'],axis=1)
X.columns

In [ ]:
#oversampling and undersampling
s4=s.loc[s['Target']==0]
s4 = s4.sample(n=50000)#downsampling
s5 = s.loc[s['Target']==1]
s5 = s5.head(n=1000)
s4 = pd.concat([s4,s5])


In [ ]:
#oversampling
s4 = pd.concat([s4,s5])
s4 = pd.concat([s4,s5])
s4 = pd.concat([s4,s5])
s4 = pd.concat([s4,s5])
s4 = pd.concat([s4,s5])
s4 = pd.concat([s4,s5])


In [ ]:
X = pd.get_dummies(s4.drop(['Target'],axis=1))#do this when sampling
#X =pd.get_dummies(X)#do this otherwise

In [ ]:
y = s4['Target']

In [ ]:
from sklearn.model_selection import train_test_split
# Random data testing

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=56)

In [ ]:
#oversampling the train data # another way to oversample
#uncomment to try
#X_train['Target'] = y_train
#X_train_2 = pd.concat([X_train,X_train_1])
#X_train_2 = pd.concat([X_train_2,X_train_1])

In [ ]:
#X_train_2 = pd.concat([X_train_2,X_train_1])
#X_train_2 = pd.concat([X_train_2,X_train_1])
#X_train_2 = pd.concat([X_train_2,X_train_1])
#X_train_2 = pd.concat([X_train_2,X_train_1])
#X_train_2 = pd.concat([X_train_2,X_train_1])

In [ ]:
#X_train_2 = X_train_2.sample(n=50000)
#X_train = X_train_2.drop(['Target'],axis=1)

In [ ]:
#y_train= X_train_2['Target']

In [ ]:
#logistic regression
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(solver = 'lbfgs')


In [ ]:
logisticRegr.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import f1_score,accuracy_score
#logisticRegr.score(test_img, y_test)
y_pred = logisticRegr.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
#scaling the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:

from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(200,), max_iter=100,learning_rate = 'adaptive',solver='sgd')
mlp.fit(X_train, y_train.values.ravel())

In [ ]:
pred_mlp = mlp.predict(X_test)
accuracy_score(y_test,pred_mlp)

In [ ]:
#to find the best paramerters
mlp = MLPClassifier(max_iter=1000)
parameter_space = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
clf.fit(X_train, y_train)


print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [ ]:
#plotting the confusion martrix

from sklearn.metrics import confusion_matrix,accuracy_score

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve,auc,r2_score
sns.heatmap(confusion_matrix(y_test,y_pred),annot=True,fmt = "d",linecolor="k",linewidths=2)
plt.title("CONFUSION MATRIX",fontsize=20)

In [ ]:
#Plottting the ROC curve
predicting_probabilites = mlp.predict_proba(X_test)[:,1]


fpr,tpr,thresholds = roc_curve(y_test,predicting_probabilites)
plt.plot(fpr,tpr,label = ("Area_under the curve :",auc(fpr,tpr)),color = "r")
plt.plot([1,0],[1,0],linestyle = "dashed",color ="k")
plt.legend(loc = "best")
plt.title("ROC - CURVE & AREA UNDER CURVE",fontsize=20)

In [ ]:
#Since denial rate is 1%, we tried with a threshold around 1% to get a rough estimate
import numpy as np
THRESHOLD = 0.015
preds = np.where(logisticRegr.predict_proba(X_test)[:,1] > THRESHOLD, 1, 0)

In [ ]:

from sklearn.metrics import classification_report
print('Results on the test set:')
print(classification_report(y_test, y_pred))

In [ ]:
#performing PCA 
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca=PCA(0.8)
pca.fit(X_train)
pca.n_components_

In [ ]:
#PCA transformation
X_train_1 = pca.transform(X_train)
X_test_1 = pca.transform(X_test)

In [ ]:
#performing SVC after PCA
from sklearn.svm import SVC
svclassifer7 = SVC(C=10,kernel='rbf',gamma='scale',probability=True)
svclassifer7.fit(X_train_1,y_train)

In [ ]:
pred_svc7 = svclassifer7.predict(X_test_1)

In [ ]:
#Importing the Keras libraries and packages
#Tried on tensorflow/keras
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
import numpy as np
y4 = np.array(y_train)

In [ ]:
model = Sequential()
from tensorflow.keras.optimizers import SGD
opt = SGD(lr=0.001)
# the shape of one training example is

model.add(Dense(units=60, activation='sigmoid',kernel_initializer='glorot_uniform', input_shape=(107,)))

model.add(Dense(units=50, activation='sigmoid'))
model.add(Dense(units=30, activation='relu'))

model.add(Dense(units=10, activation='relu'))
#model.add(Dense(units=5, activation='sigmoid'))

model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
model.fit(X_train, y4, epochs=100,batch_size=200,validation_split = 0.2)

In [ ]:
pred_nn = model.predict(X_test) 

In [ ]:
ro = np.round(pred_nn)
#then take the model and perform ROC curve and confusion matrix